# Using DArrays for reading and manipulating data
## Parallel Workshop JuliaCon 2016

In [1]:
# Add processes
addprocs(8)
# Use package for distributed arrays
using DistributedArrays

SOME GENERAL INTRODUCTION TO DARRAYs that I'm worknig on now

In [5]:
# Save the path to the data directory and load a list of subdirectories with the data
pth = "/data/MIMICII"
dirs = filter(isdir, map(t -> joinpath(pth, t), readdir(pth)))

22-element Array{UTF8String,1}:
 "/data/MIMICII/s00124"
 "/data/MIMICII/s00177"
 "/data/MIMICII/s00184"
 "/data/MIMICII/s00283"
 "/data/MIMICII/s00308"
 "/data/MIMICII/s00377"
 "/data/MIMICII/s00408"
 "/data/MIMICII/s00491"
 "/data/MIMICII/s00521"
 "/data/MIMICII/s00565"
 "/data/MIMICII/s00593"
 "/data/MIMICII/s00618"
 "/data/MIMICII/s00631"
 "/data/MIMICII/s00634"
 "/data/MIMICII/s00668"
 "/data/MIMICII/s00682"
 "/data/MIMICII/s00700"
 "/data/MIMICII/s00719"
 "/data/MIMICII/s00787"
 "/data/MIMICII/s00871"
 "/data/MIMICII/s00985"
 "/data/MIMICII/s00992"

In [6]:
# Extract the data files
fls = mapreduce(t -> map(s -> joinpath(t, s), readdir(t)), vcat, dirs)

1875-element Array{UTF8String,1}:
 "/data/MIMICII/s00124/3255538_0001.mat"
 "/data/MIMICII/s00124/3255538_0002.mat"
 "/data/MIMICII/s00124/3255538_0003.mat"
 "/data/MIMICII/s00124/3255538_0004.mat"
 "/data/MIMICII/s00124/3255538_0005.mat"
 "/data/MIMICII/s00124/3255538_0006.mat"
 "/data/MIMICII/s00124/3526846_0001.mat"
 "/data/MIMICII/s00124/3526846_0002.mat"
 "/data/MIMICII/s00124/3526846_0003.mat"
 "/data/MIMICII/s00124/3526846_0004.mat"
 "/data/MIMICII/s00124/3526846_0005.mat"
 "/data/MIMICII/s00124/3526846_0006.mat"
 "/data/MIMICII/s00124/3526846_0007.mat"
 ⋮                                      
 "/data/MIMICII/s00992/3108494_0187.mat"
 "/data/MIMICII/s00992/3108494_0188.mat"
 "/data/MIMICII/s00992/3108494_0189.mat"
 "/data/MIMICII/s00992/3108494_0190.mat"
 "/data/MIMICII/s00992/3108494_0191.mat"
 "/data/MIMICII/s00992/3108494_0192.mat"
 "/data/MIMICII/s00992/3108494_0193.mat"
 "/data/MIMICII/s00992/3108494_0194.mat"
 "/data/MIMICII/s00992/3108494_0195.mat"
 "/data/MIMICII/s00992/

In [7]:
# Size if GB
@time sum(map(filesize, fls))/1024^3

  0.037466 seconds (19.67 k allocations: 1.062 MB)


23.212903790175915

In [8]:
flsSmall = fls[1:div(length(fls), 10)]
@time sum(map(filesize, flsSmall))/1024^3

  0.000797 seconds (382 allocations: 25.172 KB)


1.6282551661133766

In [9]:
# Use package for reading binary files (written in Julia)
using MAT

In [11]:
@time dt = map(matread, distribute(flsSmall));

  0.409801 seconds (29.13 k allocations: 1.614 MB)


In [12]:
@show typeof(dt)
dt[1]

typeof(dt) = DistributedArrays.DArray{Dict{ASCIIString,Any},1,Array{Dict{ASCIIString,Any},1}}


Dict{ASCIIString,Any} with 3 entries:
  "A"      => Dict{ASCIIString,Any}("A"=>1x17 sparse matrix with 17 Float64 ent…
  "tm"     => 345000x1 Array{Float64,2}:…
  "signal" => 345000x1 Array{Float64,2}:…

In [ ]:
# Plot a signal
using PlotlyJS
plot(PlotlyJS.scattergl(;y = dt[1]["signal"][:]))

In [13]:
# custom cleaner functions are fast in Julia
# in this exmaple, replace NaNs with zeros
x = map(t -> ifelse(isnan(t), 0, t),dt[1]["signal"][:]);

# fft
xfft = fft(x)

345000-element Array{Complex{Float64},1}:
  13341.5+0.0im    
  4149.56-13970.2im
  -2368.8-1152.79im
  5407.77+874.87im 
  7131.39-5189.44im
 -530.063-9487.38im
 -6983.82+605.086im
  5317.84+6670.13im
   9060.2-6496.53im
 -3092.74-7535.38im
 -2208.69+2028.72im
  5152.03+142.519im
  4084.85-6764.49im
         ⋮         
  4084.85+6764.49im
  5152.03-142.519im
 -2208.69-2028.72im
 -3092.74+7535.38im
   9060.2+6496.53im
  5317.84-6670.13im
 -6983.82-605.086im
 -530.063+9487.38im
  7131.39+5189.44im
  5407.77-874.87im 
  -2368.8+1152.79im
  4149.56+13970.2im

In [15]:
# size in GB
@time mapreduce(Base.summarysize, +, dt)/1024^3

  0.006456 seconds (2.94 k allocations: 235.547 KB)


1.6282322583720088

In [16]:
@time lngs = map(t -> length(t["signal"]), dt)

  0.051641 seconds (7.61 k allocations: 540.323 KB)


187-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
   345000
     5978
  2679022
  8100000
   367500
   952500
  1807500
     4602
   145136
      768
   163584
   328464
      512
        ⋮
 11055000
   418250
      246
     1272
        8
     7474
     1478
        8
     1280
      768
    11000
        8

In [ ]:
plot(histogram(x = lngs), Layout(yaxis=Dict("type" => "log")))

In [22]:
using TSVD

@everywhere function Base.hcat{T<:DistributedArrays.DVector}(x::Vector{T})
    l    = length(x)
    if l == 0
        throw(ArgumentError("cannot flatten empty vector"))
    else
        x1   = x[1]
        m, n = size(x1, 1), size(x1, 2)
        B    = DArray((m, l*n)) do I
            B_local = Array(eltype(x1), map(length, I))
            for j = 1:length(I[2])
                B_local[:, j] = x[I[2][j]][I[1]]
            end
            return B_local
        end
        return B
    end
end

@everywhere Base.procs(A::Array) = fill(myid(), 1, 1)

Base.convert{S,T,N,D<:DArray}(::Type{Array{S,N}}, s::SubArray{T,N,D}) = begin
    I = s.indexes
    d = s.parent
#     println("Hej", isa(I,Tuple{Vararg{UnitRange{Int}}}))
#     if isa(I,Tuple{Vararg{UnitRange{Int}}}) && S<:T && T<:S
    l = DistributedArrays.locate(d, map(first, I)...)
        if isequal(d.indexes[l...], I)
            # SubDArray corresponds to a chunk
        return DistributedArrays.chunk(d, l...)
        end
#     end
#     a = Array(S, size(s))
#     a[[1:size(a,i) for i=1:N]...] = s
#     return a
end

convert (generic function with 547 methods)

In [17]:
# Necesarry to define our own rep function because Julia's repeat is dead slow right now 
@everywhere function rep(x::Vector, l)
    y = similar(x, l)
    cx = cycle(x)
    s = start(cx)
    @inbounds @simd for i = 1:l
        (yi, s) = next(cx, s)
        y[i] = ifelse(isnan(yi), zero(yi), yi)
    end
    return y
end

In [18]:
@time dt1 = let n = 50000
    map(t -> rep(vec(t["signal"]), n), dt);
end;

  0.259672 seconds (32.09 k allocations: 1.600 MB, 4.17% gc time)


In [20]:
# Define method for converting distributed vector of vectors to distributed matrix
function Base.hcat{T}(A::DistributedArrays.DArray{Vector{T}})
    n = length(A[1])
    D = DArray((n, length(A)), A.pids, [1, length(A.pids)]) do I
        mB, nB = map(length, I)
        # Create new DArray from the distributed vector of vectors.
        # For now, we assume that each vector is only located on a single
        # worker. Eventually, we'd like to find a more flexible solution.
        B = Array(eltype(A[1]), mB, nB)
        for i = 1:nB
            B[:,i] = A[I[2][i]][I[1]]
        end
        B
    end
    return D
end

hcat (generic function with 15 methods)

In [21]:
@time A = hcat(dt1);

  1.257840 seconds (431.32 k allocations: 19.012 MB, 0.40% gc time)


In [ ]:
# @time B = mapslices(fft, A, 1)
fft(A, 1)

In [ ]:
v0 = DArray(I -> rand(Complex64, length(I[1])), (size(A, 1),), A.pids[:,1])
@time U, s, V = TSVD.tsvd(A, 5, initVec = v0, stepSize = 5, debug = true);

τ = 0.007648707318601342
Reorth v
τ = 0.007648707318601342
Reorth u
τ = 0.007648707318601342
Reorth v
τ = 0.007648707318601342
Reorth u
